In [ ]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.svm import SVR
import lightgbm as lgb
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold,cross_validate
from xgboost import XGBRegressor as xc
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss

In [ ]:
## read trainset
df=pd.read_csv('../../data/features_trainset_NoZrAlCu3.csv')
del df['Unnamed: 0']
del df['pretty_formula']
print('data shape：',df.shape)
df.drop_duplicates(subset=['composition'], inplace=True)
print('data shape：',df.shape)
df=df.dropna(axis=1)
df=df.dropna(axis=0)
col_list=df.columns
col_name=col_list[2:len(col_list)]
featur=df[col_name].values
target=df['D_max'].values
X=featur
Y=target
del df['composition']
print('data shape：',df.shape)
df.head()

In [ ]:
dddf=pd.read_csv('../../data/features_ZrAlCu.csv')
del dddf['Unnamed: 0']
del dddf['pretty_formula']
#del ddf['composition']
print(dddf.shape)
for i in range(0,len(dddf)):
    if dddf['composition'][i]=='Zr20 Al20 Cu60':
        print(i)
    if dddf['composition'][i]=='Zr20 Al25 Cu55':
        print(i)
    if dddf['composition'][i]=='Zr40 Al20 Cu40':
        print(i)
    if dddf['composition'][i]=='Zr45 Al20 Cu35':
        print(i)
    if dddf['composition'][i]=='Zr50 Al5 Cu45':
        print(i)
    if dddf['composition'][i]=='Zr55 Al5 Cu40':
        print(i)
dddf

In [ ]:
df_2 = dddf[1855:1856][col_name]
df_2 = df_2.append(dddf[1850:1851][col_name],ignore_index=True)
df_2 = df_2.append(dddf[3280:3281][col_name],ignore_index=True)
df_2 = df_2.append(dddf[3575:3576][col_name],ignore_index=True)
df_2 = df_2.append(dddf[3830:3831][col_name],ignore_index=True)
df_2 = df_2.append(dddf[4075:4076][col_name],ignore_index=True)
df_2['D_max']=[0,0,0.2,0.2,2,1.5]
df_2

In [ ]:
df = df.append(df_2,ignore_index=True)
print(df.shape)
df

In [ ]:
##normalization
y = df['D_max']
X = df[col_name]

scaler=preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled)
X_scaled.shape

In [ ]:
##RF
param_grid = {
    'n_estimators':hp.quniform('n_estimators',50,550,5),
    'max_features':hp.quniform('max_features',3,29,1),
    'max_depth':hp.quniform('max_depth',8,55,1),
    'min_samples_split':hp.quniform('min_samples_split',2,10,1),
    'min_impurity_decrease':hp.quniform("min_impurity_decrease",0,5,0.1)
}

def hyperopt_objective(params):
    reg = RFR(n_estimators=int(params['n_estimators']),
             max_depth=int(params['max_depth']),
             max_features=int(params['max_features']),
             min_samples_split=int(params['min_samples_split']),
             min_impurity_decrease=params['min_impurity_decrease'],
             random_state = 12138,
             verbose = False,
             n_jobs=10)
    cv = KFold(n_splits=5,shuffle=True,random_state=12138)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='neg_root_mean_squared_error',cv=cv,
                                    verbose=False,n_jobs=10,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))


def param_hyperopt(max_evals=100):
    trials = Trials()
    early_stop_fn = no_progress_loss(100)
    params_best =fmin(hyperopt_objective,
                     space = param_grid,
                     algo = tpe.suggest,
                     max_evals = max_evals,
                     verbose = True,
                     trials = trials,
                     early_stop_fn = early_stop_fn
                     )
    print('\n','best params:',params_best,'\n')
    return params_best,trials

def hyperopt_validation(params):
    reg = RFR(n_estimators=int(params['n_estimators']),
             max_depth=int(params['max_depth']),
             max_features=int(params['max_features']),
             min_samples_split=int(params['min_samples_split']),
             min_impurity_decrease=int(params['min_impurity_decrease']),
             random_state =12138,
             verbose = False,
             n_jobs=-1)
    cv = KFold(n_splits=5,shuffle=True,random_state=2)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='r2',cv=cv,
                                    verbose=False,n_jobs=-1,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

 46%|█████████████████████▎                        | 232/500 [13:10<15:13,  3.41s/trial, best loss: 0.6560922633165779]

 best params: {'max_depth': 23.0, 'max_features': 8.0, 'min_impurity_decrease': 0.0, 'min_samples_split': 3.0, 'n_estimators': 390.0} 

0.7336058999919889

In [ ]:
##SVM
param_grid = {
    'C':hp.quniform('C',1,50,1),
    'gamma':hp.quniform('gamma',0.1,0.45,0.005),
    'epsilon':hp.quniform('epsilon',0,0.2,0.002)
}

def hyperopt_objective(params):
    reg = SVR(C=int(params['C']),
             epsilon=params['epsilon'],
             gamma=params['gamma'],
             kernel='rbf',
             verbose = False
             )
    cv = KFold(n_splits=5,shuffle=True,random_state=12138)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='neg_root_mean_squared_error',cv=cv,
                                    verbose=False,n_jobs=-1,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))

def param_hyperopt(max_evals=100):
    trials = Trials()
    early_stop_fn = no_progress_loss(100)
    params_best =fmin(hyperopt_objective,
                     space = param_grid,
                     algo = tpe.suggest,
                     max_evals = max_evals,
                     verbose = True,
                     trials = trials,
                     early_stop_fn = early_stop_fn
                     )
    print('\n','best params:',params_best,'\n')
    return params_best,trials

def hyperopt_validation(params):
    reg = SVR(C=int(params['C']),
             epsilon=params['epsilon'],
             gamma=params['gamma'],
             kernel='rbf',
             verbose = False)
    cv = KFold(n_splits=5,shuffle=True,random_state=2)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='r2',cv=cv,
                                    verbose=False,n_jobs=-1,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

In [ ]:
params_best,trials = param_hyperopt(500)
hyperopt_validation(params_best)

80%|█████████████████████████████████████▌         | 400/500 [33:16<08:19,  4.99s/trial, best loss: 0.629500149685196]

 best params: {'C': 7.0, 'epsilon': 0.04, 'gamma': 0.375} 

0.73568261064387970

In [ ]:
##XGBoost
def hyperopt_objective(params):
    reg = xc(n_estimators=int(params['n_estimators']),
             max_depth=int(params['max_depth']),
             reg_lambda=params['reg_lambda'],
             learning_rate=params['learning_rate'],
             subsample=params['subsample'],
             colsample_bytree=params['colsample_bytree'],
             colsample_bynode=params['colsample_bynode'],
             gamma = params['gamma'],
             min_child_weight=params['min_child_weight'],
             objective='reg:squarederror',
             random_state = 12138,
             n_jobs=-1)
    cv = KFold(n_splits=5,shuffle=True,random_state=12138)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='neg_root_mean_squared_error',cv=cv,
                                    verbose=False,n_jobs=-1,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))

param_grid_simple = {'n_estimators': hp.quniform("n_estimators",150,450,3)
                     ,"learning_rate": hp.quniform("learning_rate",0.05,0.3,0.002)
                     ,"colsample_bytree":hp.quniform("colsample_bytree",0.3,1,0.1)
                     ,"colsample_bynode":hp.quniform("colsample_bynode",0.1,1,0.1)
                     ,"gamma":hp.quniform("gamma",0,15,0.2)
                     ,"reg_lambda":hp.quniform("reg_lambda",0,25,0.5)
                     ,"min_child_weight":hp.quniform("min_child_weight",0,50,0.5)
                     ,"max_depth":hp.quniform("max_depth",5,45,1)
                     ,"subsample":hp.quniform("subsample",0.5,1,0.1)
                    }

def param_hyperopt(max_evals=100):

    trials = Trials()
    early_stop_fn = no_progress_loss(100)
    params_best = fmin(hyperopt_objective
                       , space = param_grid_simple
                       , algo = tpe.suggest
                       , max_evals = max_evals
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

def hyperopt_validation(params):
    reg = xc(n_estimators=int(params['n_estimators']),
             max_depth=int(params['max_depth']),
             reg_lambda=params['reg_lambda'],
             learning_rate=params['learning_rate'],
             subsample=params['subsample'],
             colsample_bytree=params['colsample_bytree'],
             colsample_bynode=params['colsample_bynode'],
             gamma = params['gamma'],
             min_child_weight=params['min_child_weight'],
             objective='reg:squarederror',
             random_state = 12138,
             n_jobs=-1)
    cv = KFold(n_splits=5,shuffle=True,random_state=2)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='r2',cv=cv,
                                    verbose=False,n_jobs=-1,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

24%|██████████▋                                 | 122/500 [30:32<1:34:37, 15.02s/trial, best loss: 0.6087342281968906]

 
 best params:  {'colsample_bynode': 1.0, 'colsample_bytree': 0.8, 'gamma': 0.0, 'learning_rate': 0.08, 'max_depth': 17.0, 'min_child_weight': 6.0, 'n_estimators': 255.0, 'reg_lambda': 24.5, 'subsample': 0.7000000000000001} 

0.7605680694435468

In [ ]:
##LightGBM
param_grid = {
    'n_estimators':hp.quniform('n_estimators',100,800,5),
    'num_leaves':hp.quniform('num_leaves',10,400,5),
    'learning_rate':hp.quniform('learning_rate',0.1,0.5,0.02),
    'min_child_samples':hp.quniform('min_child_samples',1,40,1),
    'reg_alpha':hp.quniform("reg_alpha",0,10,0.5),
    'reg_lambda':hp.quniform("reg_lambda",0,100,2),
    'subsample':hp.quniform("subsample",0.5,1,0.1)
}

def hyperopt_objective(params):
    reg = lgb.LGBMRegressor(
             n_estimators=int(params['n_estimators']),
             num_leaves=int(params['num_leaves']),
             min_child_samples=int(params['min_child_samples']),
             learning_rate=params['learning_rate'],
             reg_alpha=params['reg_alpha'],
             reg_lambda=params['reg_lambda'], 
             subsample = params['subsample'],
             random_state = 12138,
             verbose = int(False),
             n_jobs=-1)
    cv = KFold(n_splits=5,shuffle=True,random_state=12138)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='neg_root_mean_squared_error',cv=cv,
                                    verbose=False,n_jobs=-1,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))

def param_hyperopt(max_evals=100):

    trials = Trials()
    early_stop_fn = no_progress_loss(100)
    params_best =fmin(hyperopt_objective,
                     space = param_grid,
                     algo = tpe.suggest,
                     max_evals = max_evals,
                     verbose = True,
                     trials = trials,
                     early_stop_fn = early_stop_fn
                     )
    print('\n','best params:',params_best,'\n')
    return params_best,trials

def hyperopt_validation(params):
    reg = lgb.LGBMRegressor(
             n_estimators=int(params['n_estimators']),
             num_leaves=int(params['num_leaves']),
             min_child_samples=int(params['min_child_samples']),
             learning_rate=params['learning_rate'],
             reg_alpha=params['reg_alpha'],
             reg_lambda=params['reg_lambda'], 
             subsample = params['subsample'],
             random_state = 12138,
             verbose = int(False),
             n_jobs=-1)
    cv = KFold(n_splits=5,shuffle=True,random_state=2)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='r2',cv=cv,
                                    verbose=False,n_jobs=-1,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

In [ ]:
params_best,trials = param_hyperopt(500) 
hyperopt_validation(params_best)

33%|███████████████▎                              | 167/500 [08:45<17:27,  3.15s/trial, best loss: 0.6110591870390076]

 best params: {'learning_rate': 0.14, 'min_child_samples': 23.0, 'n_estimators': 550.0, 'num_leaves': 155.0, 'reg_alpha': 0.0, 'reg_lambda': 90.0, 'subsample': 0.7000000000000001} 

0.7598072884186303

___

In [ ]:
##Fusion
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split
import ternary
X_train,X_test,Y_train,Y_test = train_test_split(X_scaled,y,train_size=0.8,random_state=12138)

reg_svm = SVR(C=7,epsilon=0.04,gamma=0.375)
reg_rf =RFR(max_depth=23,max_features=8,min_impurity_decrease=0,min_samples_split=3,n_estimators=390,random_state=12138)
reg_xgb = xc(colsample_bynode=1,colsample_bytree=0.8,gamma=0,learning_rate=0.08,max_depth=17,
         min_child_weight=6,n_estimators=255,reg_lambda=24.5,subsample=0.7,random_state=12138)
reg_gbm =lgb.LGBMRegressor(learning_rate = 0.14,min_child_samples = 23,n_estimators =550, 
        num_leaves = 155,reg_alpha =0,reg_lambda =90,subsample = 0.7,random_state=12138)
estimators = [('svm',reg_svm), ('rf',reg_rf), ('xgb',reg_xgb), ('lightgbm',reg_gbm)]

In [ ]:
VC_hard =VotingRegressor(estimators).fit(X_train, Y_train)
print('Test',VC_hard.score(X_test,Y_test))
print('Train',VC_hard.score(X_train,Y_train))

reg_rf.fit(X_train, Y_train)
reg_xgb.fit(X_train, Y_train)
reg_svm.fit(X_train, Y_train)
reg_gbm.fit(X_train, Y_train)

print('SVM',reg_svm.score(X_train, Y_train),reg_svm.score(X_test,Y_test))
print('RF',reg_rf.score(X_train, Y_train),reg_rf.score(X_test,Y_test))
print('XGB',reg_xgb.score(X_train, Y_train),reg_xgb.score(X_test,Y_test))
print('LightGBM',reg_gbm.score(X_train, Y_train),reg_gbm.score(X_test,Y_test))

In [ ]:
params_space = {
                'weight1': hp.quniform("weight1",0,1,0.01),
                'weight2': hp.quniform("weight2",0,1,0.01),
                'weight3': hp.quniform("weight3",0,1,0.01),
                'weight4': hp.quniform("weight4",0,1,0.01)
}

def hyperopt_objective_weight(params):
    weight1 = params['weight1']
    weight2 = params['weight2']
    weight3 = params['weight3']
    weight4 = params['weight4']
    weights = [weight1, weight2, weight3,weight4]
    
    reg = VotingRegressor(estimators=estimators, n_jobs=-1,weights=weights)
    cv = KFold(n_splits=5,shuffle=True,random_state=12138)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='r2',cv=cv,
                                    verbose=False,n_jobs=8,error_score='raise')
    return -np.mean(abs(validation_loss['test_score']))

def param_hyperopt(max_evals=100):
    
    trials = Trials()
    early_stop_fn = no_progress_loss(50)
    params_best = fmin(hyperopt_objective_weight
                       , space = params_space
                       , algo = tpe.suggest
                       , max_evals = max_evals
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )

    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

def hyperopt_validation(params):
    reg = VotingRegressor(estimators=estimators,n_jobs=-1,
                          weights=[params[0]['weight1'],params[0]['weight2'],params[0]['weight3'],params[0]['weight4']])
    cv = KFold(n_splits=5,shuffle=True,random_state=2)
    validation_loss = cross_validate(reg,X_scaled,y,scoring='r2',cv=cv,
                                    verbose=False,n_jobs=-1,error_score='raise')
    return np.mean(abs(validation_loss['test_score']))

In [ ]:
params_best = param_hyperopt(300)
hyperopt_validation(params_best)

In [ ]:
params_best = param_hyperopt(300)
hyperopt_validation(params_best)

In [ ]:
params_best = param_hyperopt(300)
hyperopt_validation(params_best)

70%|█████████████████████████████▉             | 209/300 [1:14:26<32:24, 21.37s/trial, best loss: -0.7908373474093446]

 
 best params:  {'weight1': 0.75, 'weight2': 0.0, 'weight3': 0.7000000000000001, 'weight4': 0.6900000000000001} 

0.7757512998599477

In [ ]:
weight1 = 0.75
weight2 = 0
weight3 = 0.7
weight4 = 0.69

weights = [weight1, weight2, weight3, weight4]

weight_sum = weight1 + weight2 + weight3 + weight4
X_train,X_test,Y_train,Y_test = train_test_split(X_scaled,y,train_size=0.8,random_state=12138)
Voting_soft_weight = VotingRegressor(estimators=estimators, n_jobs=-1,
                                      weights=weights).fit(X_train, Y_train)
print(Voting_soft_weight.score(X_train, Y_train))
print(Voting_soft_weight.score(X_test, Y_test))

In [ ]:
df_feature=pd.read_csv('../../data/features_ZrAlCu.csv')
compositions=df_feature['composition']
del df_feature['Unnamed: 0']
del df_feature['composition']
del df_feature['pretty_formula'] 
print('data shape：',df_feature.shape)
df_feature.head()

In [ ]:
pred_X = df_feature[col_name]
pred_X_transformed=scaler.transform(pred_X)
pred_X_transformed.shape

In [ ]:
weight1 = 0.75
weight2 = 0
weight3 = 0.7
weight4 = 0.69
weights = [weight1, weight2, weight3, weight4]
Voting_soft_weight = VotingRegressor(estimators=estimators, n_jobs=-1,
                                      weights=weights).fit(X_scaled, y)

weight_sum = weight1 + weight2 + weight3
xxx = Voting_soft_weight.predict(pred_X_transformed)
df_pred_ZrAlCu=pd.DataFrame({'pretty_formula':compositions,'pred_D_max':xxx})
df_pred_ZrAlCu['pred_D_max'].describe()
df_pred_ZrAlCu.to_csv('ZrAlCu_iteration3_D_max.csv')
comp_1=[]
comp_2=[]
comp_3=[]
points=[]
for i in range(0,101,1):
    for j in range(0,101-i,1):
        k=100-i-j
        comp_1.append(i)
        comp_2.append(j)
        comp_3.append(k)
        points.append((i,j,k))

D_max=df_pred_ZrAlCu['pred_D_max'].values
data=dict()
for x in range(0,len(D_max)):
    data[points[x]]=D_max[x]
    
scale=100
figure,tax = ternary.figure(scale=scale)
figure.set_size_inches((10,8))
figure.set_facecolor('w')
tax.boundary(linewidth=1.5)
tax.gridlines(color='blue',multiple=10,linewidth=0.5,alpha=0.7)
tax.ticks(axis='lbr',linewidth=1,multiple=20,fontsize=20,offset=0.02)

tax.clear_matplotlib_ticks()
tax.get_axes().axis('off')

tax.heatmap(data, scale=scale,style="h", vmin=min(D_max), vmax=max(D_max), cmap='coolwarm',use_rgba=False, colorbar=True)
tax.left_axis_label(r"$\leftarrow$ Cu", fontsize=30, offset=0.12)
tax.right_axis_label(r"$\leftarrow$ Al", fontsize=30, offset=0.12)
tax.bottom_axis_label("Zr "+r"$\rightarrow$", fontsize=30, offset=0.04)
tax.savefig('ZrAlCu_iteration3_D_max.jpg',bbox_inches='tight',dpi=300)

In [ ]:
sorted_data = np.sort(df_pred_ZrAlCu['pred_D_max'])
P = int(0.95 * len(sorted_data))
threshold = sorted_data[P]

print(f"90%阈值为：{threshold}")

D_max_criteria=threshold
points_D_max = []
points_value=[]
comp_1=[]
comp_2=[]
comp_3=[]
points=[]
for i in range(0,101,1):
    for j in range(0,101-i,1):
        k=100-i-j
        comp_1.append(i)
        comp_2.append(j)
        comp_3.append(k)
        points.append((i,j,k))
A=comp_1
B=comp_2
C=comp_3

scale=100
fontsize=30
figure,tax = ternary.figure(scale=scale)
figure.set_size_inches((10,8))
figure.set_facecolor('w')
tax.boundary(linewidth=1.5)
tax.gridlines(color='blue',multiple=10,linewidth=0.5,alpha=0.7)
tax.ticks(axis='lbr',linewidth=1,multiple=20,fontsize=20,offset=0.02)
tax.clear_matplotlib_ticks()
tax.get_axes().axis('off')

for i in range(0,len(df_pred_ZrAlCu)):
    if df_pred_ZrAlCu['pred_D_max'][i]>D_max_criteria:
        if A[i]>0 and B[i]>0 and C[i]>0:
            points_D_max.append((A[i],B[i],C[i]))
            points_value.append(df_pred_ZrAlCu['pred_D_max'][i])
        else:
            continue
    else:
        continue
        
if len(points_D_max)>0:
    D_max=points_value
    data=dict()
    for x in range(0,len(D_max)):
        data[points_D_max[x]]=D_max[x]
    
    tax.heatmap(data, scale=scale,style="h", vmin=min(D_max), vmax=max(D_max), cmap='coolwarm',use_rgba=False, colorbar=True)
    

tax.left_axis_label(r"$\leftarrow$ "+'Cu', fontsize=fontsize, offset=0.12)
tax.right_axis_label(r"$\leftarrow$ "+'Al', fontsize=fontsize, offset=0.12)
tax.bottom_axis_label('Zr'+" "+r"$\rightarrow$", fontsize=fontsize, offset=0.04)
tax.savefig('ZrAlCu+2+2+2.tif',bbox_inches='tight',dpi=300)